In [1]:
CELL_TYPE = 'pDC'
N_GENES: int = 20
SEED = 'shap_studyID' #'disease_NOstudy' 'study_NOdisease' or 'int' or 'shap_studyID'
TEST_SPLIT_IDX: int = 1 #[0,4]

In [2]:
# Parameters
CELL_TYPE = "UTC"
SEED = "shap_disease_NOstudy"
TEST_SPLIT_IDX = 0


In [3]:
N_SPLITS: int = 5
N_TRIALS: int = 50

In [4]:
import os
import sys
from pyprojroot.here import here
import pandas as pd
import anndata as ad
import numpy as np
import math
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
from sklearn.metrics import balanced_accuracy_score, f1_score
import optuna

import joblib
import pickle
import datetime

import collections

import xgboost
from sklearn.preprocessing import LabelEncoder

import scipy.sparse as ssp
import joblib

from dotenv import load_dotenv

In [5]:
load_dotenv()

True

# LOAD DATASET

In [6]:
train_adata = ad.read_h5ad(
    here(f'03_downstream_analysis/08_gene_importance/xgboost_external_validation/xgboost_TopN_genes/data_cellTypes/EXTERNAL_{CELL_TYPE}.filtered.log1p.h5ad')
)

In [7]:
if SEED != 'all':
    gene_subset = np.load(here(f'03_downstream_analysis/08_gene_importance/xgboost_external_validation/shap_gene_selection/gene_subsets_{N_GENES}/{CELL_TYPE}_{SEED}.npy'), allow_pickle=True)
    train_adata = train_adata[:,gene_subset]
    print(gene_subset)
elif SEED == 'all':
    print('Using all genes')
else:
    raise ValueError()

['ENSG00000002586' 'ENSG00000005339' 'ENSG00000005844' 'ENSG00000007264'
 'ENSG00000015475' 'ENSG00000027697' 'ENSG00000030110' 'ENSG00000030582'
 'ENSG00000035115' 'ENSG00000051108' 'ENSG00000066136' 'ENSG00000069399'
 'ENSG00000071073' 'ENSG00000075624' 'ENSG00000076944' 'ENSG00000077380'
 'ENSG00000078596' 'ENSG00000089692' 'ENSG00000099624' 'ENSG00000100280'
 'ENSG00000101439' 'ENSG00000104660' 'ENSG00000104671' 'ENSG00000104763'
 'ENSG00000104904' 'ENSG00000105374' 'ENSG00000105835' 'ENSG00000107223'
 'ENSG00000107968' 'ENSG00000108622' 'ENSG00000110848' 'ENSG00000112149'
 'ENSG00000113732' 'ENSG00000114737' 'ENSG00000115415' 'ENSG00000115523'
 'ENSG00000116171' 'ENSG00000117602' 'ENSG00000117984' 'ENSG00000120837'
 'ENSG00000123268' 'ENSG00000125347' 'ENSG00000125534' 'ENSG00000125735'
 'ENSG00000125740' 'ENSG00000126561' 'ENSG00000127022' 'ENSG00000127152'
 'ENSG00000127184' 'ENSG00000130066' 'ENSG00000130429' 'ENSG00000130522'
 'ENSG00000132475' 'ENSG00000132965' 'ENSG000001336

In [8]:
train_adata.shape

(14828, 130)

In [9]:
train_adata.obs.disease.unique()

['RA', 'COVID', 'healthy', 'HIV', 'cirrhosis', 'CD', 'SLE', 'sepsis']
Categories (8, object): ['CD', 'COVID', 'HIV', 'RA', 'SLE', 'cirrhosis', 'healthy', 'sepsis']

In [10]:
train_adata.obs.sampleID.unique()

['SCGT00val_I0364_T0', 'SCGT00val_I036016_T0', 'SCGT00val_I036021_T0', 'SCGT00val_I036015_T0', 'SCGT00val_I036023_T0', ..., '10XGenomics_10XHC2_T0', '10XGenomics_10XHC3_T0', '10XGenomics_10XHC5_T0', '10XGenomics_10XHC7_T0', '10XGenomics_10XHC8_T0']
Length: 86
Categories (86, object): ['10XGenomics_10XHC1_T0', '10XGenomics_10XHC2_T0', '10XGenomics_10XHC3_T0', '10XGenomics_10XHC4_T0', ..., 'Savage2021_BRISL6_T0', 'Savage2021_BRISL7_T0', 'Savage2021_PIDA_T0', 'Savage2021_PIDB_T0']

In [11]:
all_idxs = np.arange(train_adata.obs.shape[0])
left_out_splits = [s[1] for s in StratifiedGroupKFold(n_splits=N_SPLITS).split(all_idxs, train_adata.obs.disease, train_adata.obs.sampleID)]

In [12]:
TRAIN_SPLIT_IDXS = [0,1,2,3,4]
VAL_SPLIT_IDX = (TEST_SPLIT_IDX + 1) % 5
TRAIN_SPLIT_IDXS.remove(TEST_SPLIT_IDX)
TRAIN_SPLIT_IDXS.remove(VAL_SPLIT_IDX)
TRAIN_SPLIT_IDXS, VAL_SPLIT_IDX, TEST_SPLIT_IDX

([2, 3, 4], 1, 0)

In [13]:
train_idxs = np.concatenate([left_out_splits[idx] for idx in TRAIN_SPLIT_IDXS])
val_idxs = left_out_splits[VAL_SPLIT_IDX]
test_idxs = left_out_splits[TEST_SPLIT_IDX]

### SUBSET DATASET INTO TRAIN/TEST/VAL SPLITS

In [14]:
X_train = train_adata.X[train_idxs]
X_test = train_adata.X[test_idxs]
X_val = train_adata.X[val_idxs]
X_train.shape, X_test.shape, X_val.shape

((9125, 130), (2840, 130), (2863, 130))

In [15]:
y_train = train_adata.obs.iloc[train_idxs].disease.values.astype(str)
y_test = train_adata.obs.iloc[test_idxs].disease.values.astype(str)
y_val = train_adata.obs.iloc[val_idxs].disease.values.astype(str)
y_train.shape, y_test.shape, y_val.shape

((9125,), (2840,), (2863,))

In [16]:
lenc = LabelEncoder()
y_train_enc = lenc.fit_transform(y_train)
y_val_enc = lenc.transform(y_val)
y_test_enc = lenc.transform(y_test)

### GENERATE F1 

In [17]:
def custom_f1_score(y_true, y_pred):
    return -f1_score(y_true, y_pred.argmax(1), average='weighted')

In [18]:
eval_metric=custom_f1_score
eval_metric_name='custom_f1_score'

def objective(trial):
    params = {
        'n_estimators': 1500,
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 250),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 5e-1, log=True),
    }
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, f'validation_0-{eval_metric_name}')
    es_callback = xgboost.callback.EarlyStopping(20, min_delta=0.001)
    xgb = xgboost.XGBClassifier(
        eval_metric=eval_metric,
        callbacks=[pruning_callback, es_callback],
        n_jobs=5,
        **params
    )
    xgb.fit(
        X_train, 
        y_train_enc, 
        verbose=0,
        eval_set=[(X_val, y_val_enc)],
    )
    trial.set_user_attr('best_iteration', xgb.best_iteration)

    return xgb.best_score

In [19]:
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=N_TRIALS, gc_after_trial=True)

[I 2025-05-14 18:09:12,549] A new study created in memory with name: no-name-3e0968a7-a7ce-4e6e-9e5b-2a01d33756ef


[I 2025-05-14 18:09:13,294] Trial 0 finished with value: -0.587156 and parameters: {'max_depth': 9, 'min_child_weight': 238, 'subsample': 0.7587945476302645, 'colsample_bynode': 0.6387926357773329, 'learning_rate': 0.0026368755339723046}. Best is trial 0 with value: -0.587156.


[I 2025-05-14 18:09:20,289] Trial 1 finished with value: -0.75374 and parameters: {'max_depth': 5, 'min_child_weight': 15, 'subsample': 0.8795585311974417, 'colsample_bynode': 0.6410035105688879, 'learning_rate': 0.08148293210105287}. Best is trial 1 with value: -0.75374.


[I 2025-05-14 18:09:21,094] Trial 2 finished with value: -0.610857 and parameters: {'max_depth': 3, 'min_child_weight': 243, 'subsample': 0.8491983767203796, 'colsample_bynode': 0.29110519961044856, 'learning_rate': 0.003095566460242371}. Best is trial 1 with value: -0.75374.


[I 2025-05-14 18:09:21,906] Trial 3 finished with value: -0.638798 and parameters: {'max_depth': 6, 'min_child_weight': 77, 'subsample': 0.5722807884690141, 'colsample_bynode': 0.48875051677790415, 'learning_rate': 0.006109683510122491}. Best is trial 1 with value: -0.75374.


...

[I 2025-05-14 18:10:39,224] Trial 45 finished with value: -0.768018 and parameters: {'max_depth': 9, 'min_child_weight': 12, 'subsample': 0.9430855867576404, 'colsample_bynode': 0.22949990797584902, 'learning_rate': 0.2566551637564491}. Best is trial 18 with value: -0.773275.


[I 2025-05-14 18:10:39,447] Trial 46 pruned. Trial was pruned at iteration 0.


[I 2025-05-14 18:10:39,659] Trial 47 pruned. Trial was pruned at iteration 0.


[I 2025-05-14 18:10:39,861] Trial 48 pruned. Trial was pruned at iteration 0.


[I 2025-05-14 18:10:40,060] Trial 49 pruned. Trial was pruned at iteration 0.


In [20]:
out_dir = here(f'03_downstream_analysis/08_gene_importance/xgboost_external_validation/xgboost_TopN_genes/results_{N_GENES}/study')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
joblib.dump(study,os.path.join(out_dir, f'{CELL_TYPE}_{SEED}_{TEST_SPLIT_IDX}_xgboost.pkl'))

['/scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/03_downstream_analysis/08_gene_importance/xgboost_external_validation/xgboost_TopN_genes/results_20/study/UTC_shap_disease_NOstudy_0_xgboost.pkl']

In [21]:
n_estimators = int(study.best_trial.user_attrs['best_iteration']*1.2)
xgb = xgboost.XGBClassifier(
        eval_metric=eval_metric,
        n_estimators=n_estimators,
        **study.best_trial.params
    )
xgb.fit(
    ssp.vstack((X_train, X_val)), 
    np.concatenate((y_train_enc, y_val_enc)),
    verbose=1,
)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=0.24900065324684006,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False,
              eval_metric=<function custom_f1_score at 0x7f9bbc10b920>,
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.16384379577990202, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=11, max_leaves=None,
              min_child_weight=37, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=188, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [22]:
out_dir = here(f'03_downstream_analysis/08_gene_importance/xgboost_external_validation/xgboost_TopN_genes/results_{N_GENES}/best_model')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
joblib.dump(xgb, os.path.join(out_dir, f'{CELL_TYPE}_{SEED}_{TEST_SPLIT_IDX}_xgb.json'))

['/scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/03_downstream_analysis/08_gene_importance/xgboost_external_validation/xgboost_TopN_genes/results_20/best_model/UTC_shap_disease_NOstudy_0_xgb.json']

In [23]:
df_pred_test = pd.DataFrame(dict(
    cell_id=train_adata.obs.iloc[test_idxs].index.values,
    y_true=y_test, 
    y_true_code=y_test_enc, 
    y_pred=xgb.predict(X_test))).set_index('cell_id')

In [24]:
out_dir = here(f'03_downstream_analysis/08_gene_importance/xgboost_external_validation/xgboost_TopN_genes/results_{N_GENES}/predictions')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
df_pred_test.to_csv(os.path.join(out_dir, f'{CELL_TYPE}_{SEED}_{TEST_SPLIT_IDX}_pred_test.zip'))

In [25]:
metrics_dict = dict(
    BAS=balanced_accuracy_score(y_true=df_pred_test.y_true_code, y_pred=df_pred_test.y_pred), WF1=f1_score(y_true=df_pred_test.y_true_code, y_pred=df_pred_test.y_pred,average='weighted'))

In [26]:
metrics_dict

{'BAS': 0.47406182155948906, 'WF1': 0.7446305721459373}

In [27]:
metrics_df = pd.DataFrame.from_dict([metrics_dict]).assign(split_idx=TEST_SPLIT_IDX, gene_set_seed=SEED, cell_type=CELL_TYPE)
metrics_df

,BAS,WF1,split_idx,gene_set_seed,cell_type
0,0.474062,0.744631,0,shap_disease_NOstudy,UTC


In [28]:
out_dir = here(f'03_downstream_analysis/08_gene_importance/xgboost_external_validation/xgboost_TopN_genes/results_{N_GENES}/metrics')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
metrics_df.to_csv(os.path.join(out_dir, f'{CELL_TYPE}_{SEED}_{TEST_SPLIT_IDX}_metrics.zip'))